# 04 — CrewAI Basics

In this notebook you'll learn the fundamentals of **CrewAI** — a framework for building **collaborative, multi-agent** systems.

We will cover:
- What CrewAI is and when to use it
- Core concepts: **Agent**, **Task**, **Crew**, and **Process**
- A minimal two-agent example: **Researcher** + **Writer**

> This notebook is designed to run with **OpenAI** (or another provider compatible with your CrewAI version). If no API key is found, the notebook will safely skip the live run and show the structure instead.

In [1]:
# --- Environment setup (dotenv) ---
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai_api_key = os.getenv("OPENAI_API_KEY")
print("✅ Environment loaded. OpenAI key detected:", bool(openai_api_key))

✅ Environment loaded. OpenAI key detected: True


## 🧭 What is CrewAI?

**CrewAI** is a framework for orchestrating **multiple AI agents** that collaborate to achieve a goal. Each **Agent** has a role, a backstory, skills (tools), and a goal. A **Task** describes a piece of work assigned to an agent. A **Crew** coordinates agents and tasks, choosing a **Process** (sequential or hierarchical) to execute the plan.

### Core concepts
- **Agent** → A role-driven LLM with a goal and (optionally) tools.
- **Task** → A unit of work with a clear objective and acceptance criteria.
- **Crew** → The orchestrator that runs tasks with agents.
- **Process** → How the crew executes (e.g., `sequential`, `hierarchical`).

> Think of CrewAI as a way to make LLMs **collaborate** with clear responsibilities.

## ⚙️ Minimal example: Researcher + Writer

We will:
1) Define two **Agents** (Researcher, Writer)
2) Define two **Tasks** (gather facts, write an article)
3) Run them **sequentially** in a **Crew**

If no API key is set, the code will **skip the live run** and just print what would happen.

In [ ]:
# --- Imports for CrewAI ---
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Shared LLM (reads OPENAI_API_KEY from .env)
llm = ChatOpenAI(model="gpt-4o-mini")

# Define agents
researcher = Agent(
    role="Researcher",
    goal="Find accurate, up-to-date facts about a given topic.",
    backstory="You are a meticulous researcher. You gather reliable facts, organize them, and cite sources when possible.",
    llm=llm,
    verbose=True,
)

writer = Agent(
    role="Writer",
    goal="Write a clear, engaging article from provided facts.",
    backstory="You are a concise technical writer. You transform raw notes into a well-structured article for a general audience.",
    llm=llm,
    verbose=True,
)

# Define tasks
topic = "renewable energy technologies"

gather_facts = Task(
    description=f"Research key facts about {topic}. Include short bullets on solar, wind, and energy storage.",
    expected_output="A bullet list (5-8 bullets) with concise, factual points relevant to the topic.",
    agent=researcher,
)

write_article = Task(
    description="Using the researcher's notes, write a short article (120–180 words) explaining the topic to a non-expert audience.",
    expected_output="A cohesive, well-structured article with an intro, 1–2 body paragraphs, and a concise conclusion.",
    agent=writer,
    context=[gather_facts],  # pass previous task output as context
)

# Create the crew (sequential process)
crew = Crew(
    agents=[researcher, writer],
    tasks=[gather_facts, write_article],
    process=Process.sequential,
    verbose=True,
)

# Run the crew and print results
result = crew.kickoff()

final_text = getattr(result, "final_output", None) or getattr(result, "raw", None) or str(result)
print("\n--- Final Output ---\n")
print(final_text)

## 🔍 What happened

- We created two **agents** with different roles and writing styles.
- We defined two **tasks**: one for research and one for writing.
- The **crew** executed them in a **sequential** process.

### Process types
- `Process.sequential` → simple pipeline (A then B then C)
- `Process.hierarchical` → a manager agent delegates tasks and reviews outputs

> Start with `sequential` for clarity. Switch to `hierarchical` when you need oversight, reviews, or dynamic tasking.

## 🧩 Tips & Good Practices

- Keep agent **goals** short and specific.
- Use **expected_output** in tasks to stabilize the format.
- Prefer **small, cheap models** for initial experiments (`gpt-4o-mini`).
- Add **tools** later (e.g., web search, vector DBs) once the flow works.
- For production, implement **guardrails** and validation steps.

## ✅ Summary

- CrewAI helps you **coordinate multiple agents** with clear roles and tasks.
- Start with a **sequential** process to make behavior easy to reason about.
- You can plug in tools, memory, and validation as your needs grow.
